In [3]:
import os
print(os.getcwd())

C:\Users\nette\Dateistruktur\RWTH\Auslandssemester\ImSemester\CS-401 Applied Data Analysis\Project\P3\the-five-data-wranglers


In [4]:
%load_ext autoreload
%autoreload 2
from config import df_ba_ratings, df_rb_ratings
from scr.models.seasonality_analysis import *
from scr.models.foreign_beer import *
from scr.models.experience_words import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
plot_beer_style_ranking_by_amount(df_rb_ratings, df_rb_ratings['style'].unique(),  300, 14)

We lost 97 rows by filtering out styles with less than 14 reviews.


# Generaing the plots for the experience_words analysis

In [5]:
exp_user_ids_ba = get_experienced_users2(df_ba_ratings, exp_words1)
df_ba_ratings_of_exp, df_ba_ratings_of_inexp = split_by_experience(df_ba_ratings, exp_user_ids_ba)

In [6]:
plot_df, most_rated = calculate_style_distribution(df_ba_ratings_of_exp, df_ba_ratings_of_inexp)

In [7]:
rating_diff_df = calculate_rating_difference(df_ba_ratings_of_exp, df_ba_ratings_of_inexp, most_rated)

In [8]:
rating_diff_df2, distr_diff_df = calculate_rating_difference_with_ci(df_ba_ratings_of_exp, df_ba_ratings_of_inexp, most_rated)

In [10]:
save_plot_combined_distribution_and_rating_difference_with_ci(plot_df, rating_diff_df2, distr_diff_df)